# Multi-Class Classification with Transformer Encoder

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import math
import torch as T
import torch.nn as nn
import pandas as pd
import numpy as np
import h5py
# import torchtext
# device = T.device("cuda" if T.cuda.is_available() else "cpu")
device = T.device('cuda:1')
T.__version__, device

('1.12.1+cu113', device(type='cuda', index=1))

## 1. Making Dataset

### 1-1. Trim & Sort

In [4]:
df = pd.read_pickle('xy_full.pkl')
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-3M-AB46-01A,G2,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
1,TCGA-3M-AB47-01A,G3,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
2,TCGA-B7-5818-01A,G3,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
3,TCGA-B7-A5TI-01A,G3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
4,TCGA-B7-A5TJ-01A,G1,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
403,TCGA-VQ-AA6J-01A,G3,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
404,TCGA-VQ-AA6K-01A,G3,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
405,TCGA-ZA-A8F6-01A,G2,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


In [10]:
df = df.iloc[:, 0:8432]
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000198089.13,ENSG00000200752.1,ENSG00000263626.1,ENSG00000222146.1,ENSG00000270813.2,ENSG00000279548.1,ENSG00000214369.2,ENSG00000243345.1,ENSG00000261294.1,ENSG00000108344.13
0,TCGA-3M-AB46-01A,G2,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,...,14.701597,11.822101,12.983848,0.0,15.991650,11.094245,12.470390,0.0,0.000000,19.743297
1,TCGA-3M-AB47-01A,G3,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,...,15.255973,0.000000,0.000000,0.0,14.692933,10.435982,0.000000,0.0,0.000000,18.458569
2,TCGA-B7-5818-01A,G3,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,...,15.589529,0.000000,0.000000,0.0,11.175611,9.311638,0.000000,0.0,11.916588,20.109258
3,TCGA-B7-A5TI-01A,G3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,...,15.179206,0.000000,0.000000,0.0,9.206942,8.926808,8.718857,0.0,9.947189,19.273892
4,TCGA-B7-A5TJ-01A,G1,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,...,15.722281,0.000000,11.754862,0.0,11.592944,9.728557,10.782298,0.0,0.000000,18.676226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,...,16.477530,0.000000,0.000000,0.0,9.060223,11.651777,0.000000,0.0,9.800365,21.555788
403,TCGA-VQ-AA6J-01A,G3,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,...,15.819668,0.000000,0.000000,0.0,12.300521,10.194837,9.113332,0.0,0.000000,19.229696
404,TCGA-VQ-AA6K-01A,G3,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,...,16.535544,0.000000,0.000000,0.0,11.736541,10.608387,10.248235,0.0,11.477852,18.816272
405,TCGA-ZA-A8F6-01A,G2,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,...,15.837475,12.623232,11.785379,0.0,12.301361,10.758215,9.813609,0.0,12.364504,18.610137


In [11]:
df.columns[2:]

Index(['ENSG00000242268.2', 'ENSG00000270112.3', 'ENSG00000167578.15',
       'ENSG00000273842.1', 'ENSG00000078237.5', 'ENSG00000146083.10',
       'ENSG00000225275.4', 'ENSG00000158486.12', 'ENSG00000198242.12',
       'ENSG00000259883.1',
       ...
       'ENSG00000198089.13', 'ENSG00000200752.1', 'ENSG00000263626.1',
       'ENSG00000222146.1', 'ENSG00000270813.2', 'ENSG00000279548.1',
       'ENSG00000214369.2', 'ENSG00000243345.1', 'ENSG00000261294.1',
       'ENSG00000108344.13'],
      dtype='object', length=8430)

In [12]:
# 유전자위치번호에서 소수점이하 버린 리스트를 만들고
gene_locations_trimmed = [gene_locations[:15] for gene_locations in df.columns[2:]]

In [13]:
# 그 리스트를 가지고 데이터프레임을 다시 만듦
for original, trimmed in zip(df.columns[2:], gene_locations_trimmed):
    df.rename(columns={original:trimmed}, inplace=True)
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000242268,ENSG00000270112,ENSG00000167578,ENSG00000273842,ENSG00000078237,ENSG00000146083,ENSG00000225275,ENSG00000158486,...,ENSG00000198089,ENSG00000200752,ENSG00000263626,ENSG00000222146,ENSG00000270813,ENSG00000279548,ENSG00000214369,ENSG00000243345,ENSG00000261294,ENSG00000108344
0,TCGA-3M-AB46-01A,G2,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,...,14.701597,11.822101,12.983848,0.0,15.991650,11.094245,12.470390,0.0,0.000000,19.743297
1,TCGA-3M-AB47-01A,G3,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,...,15.255973,0.000000,0.000000,0.0,14.692933,10.435982,0.000000,0.0,0.000000,18.458569
2,TCGA-B7-5818-01A,G3,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,...,15.589529,0.000000,0.000000,0.0,11.175611,9.311638,0.000000,0.0,11.916588,20.109258
3,TCGA-B7-A5TI-01A,G3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,...,15.179206,0.000000,0.000000,0.0,9.206942,8.926808,8.718857,0.0,9.947189,19.273892
4,TCGA-B7-A5TJ-01A,G1,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,...,15.722281,0.000000,11.754862,0.0,11.592944,9.728557,10.782298,0.0,0.000000,18.676226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,...,16.477530,0.000000,0.000000,0.0,9.060223,11.651777,0.000000,0.0,9.800365,21.555788
403,TCGA-VQ-AA6J-01A,G3,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,...,15.819668,0.000000,0.000000,0.0,12.300521,10.194837,9.113332,0.0,0.000000,19.229696
404,TCGA-VQ-AA6K-01A,G3,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,...,16.535544,0.000000,0.000000,0.0,11.736541,10.608387,10.248235,0.0,11.477852,18.816272
405,TCGA-ZA-A8F6-01A,G2,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,...,15.837475,12.623232,11.785379,0.0,12.301361,10.758215,9.813609,0.0,12.364504,18.610137


In [14]:
# 컬럼 이름 오름차순으로 데이터프레임 정렬(나중에 새로운 데이터로 추론시에 유전자위치가 섞여 있으면 안 되기 때문)
dfs = df.sort_index(axis=1)
dfs

,ENSG00000001084,ENSG00000001460,ENSG00000002745,ENSG00000003147,ENSG00000003987,ENSG00000004139,ENSG00000004478,ENSG00000004660,ENSG00000004700,ENSG00000004777,...,ENSG00000281884,ENSG00000281905,ENSG00000281920,ENSGR0000167393,ENSGR0000223511,ENSGR0000223773,ENSGR0000227159,ENSGR0000264510,neoplasm_histologic_grade,submitter_id.samples
0,17.451362,14.197870,9.557661,17.824246,11.974875,13.248805,17.250061,15.076264,16.274193,15.976285,...,13.538715,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,G2,TCGA-3M-AB46-01A
1,17.074325,14.691390,10.579867,17.416783,12.736344,13.777775,18.597290,14.349628,16.525061,14.905235,...,11.339937,0.0,10.489476,0.0,0.0,0.0,0.0,0.0,G3,TCGA-3M-AB47-01A
2,17.298747,14.667178,7.653180,15.943846,9.843945,12.366148,19.835757,12.982853,15.439344,15.947792,...,11.214414,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,G3,TCGA-B7-5818-01A
3,17.073214,15.588439,9.998846,17.072270,11.826480,14.561804,18.197390,14.460233,16.930816,15.961630,...,12.565484,0.0,8.810664,0.0,0.0,0.0,0.0,0.0,G3,TCGA-B7-A5TI-01A
4,17.343706,14.781760,9.743241,18.496318,12.552886,13.926777,19.219217,15.571513,16.131040,15.285465,...,12.631523,0.0,10.874266,0.0,0.0,0.0,0.0,0.0,G1,TCGA-B7-A5TJ-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,18.266418,15.886003,9.338112,17.735695,13.432395,13.814644,19.593376,13.129708,17.279123,15.200760,...,12.096667,0.0,8.664027,0.0,0.0,0.0,0.0,0.0,G2,TCGA-VQ-AA6G-01A
403,18.807228,15.414743,11.115848,17.555781,12.022417,14.789731,18.709658,13.141469,17.381085,15.212400,...,11.224180,0.0,8.207632,0.0,0.0,0.0,0.0,0.0,G3,TCGA-VQ-AA6J-01A
404,17.097678,14.390502,10.623079,17.754175,12.433514,13.522738,19.583999,15.219955,17.203469,17.035059,...,12.190284,0.0,10.754939,0.0,0.0,0.0,0.0,0.0,G3,TCGA-VQ-AA6K-01A
405,17.111343,14.149700,11.165029,17.502426,13.523174,15.115193,18.479489,15.079098,16.549894,15.267247,...,14.040420,0.0,9.905528,0.0,0.0,0.0,0.0,0.0,G2,TCGA-ZA-A8F6-01A


In [15]:
# 정답값이 되는 악성 종양 등급 별로 행 정렬(훈련,검증데이터로 나눌 때 필요)
dfs2 = dfs.sort_values(by='neoplasm_histologic_grade')
dfs2

,ENSG00000001084,ENSG00000001460,ENSG00000002745,ENSG00000003147,ENSG00000003987,ENSG00000004139,ENSG00000004478,ENSG00000004660,ENSG00000004700,ENSG00000004777,...,ENSG00000281884,ENSG00000281905,ENSG00000281920,ENSGR0000167393,ENSGR0000223511,ENSGR0000223773,ENSGR0000227159,ENSGR0000264510,neoplasm_histologic_grade,submitter_id.samples
318,17.741417,14.171273,11.285698,18.566679,13.264547,13.822387,18.764541,14.411721,15.706475,15.462137,...,13.087562,0.0,10.330718,0.0,0.0,0.0,0.0,0.0,G1,TCGA-KB-A93H-01A
317,16.358518,13.730371,10.385346,17.389039,13.744497,14.838198,17.894246,14.631428,17.327174,15.896477,...,12.573802,0.0,8.818965,0.0,0.0,0.0,0.0,0.0,G1,TCGA-KB-A93G-01A
132,16.668176,15.105117,10.441765,17.441566,11.878497,14.131243,19.423980,15.489673,15.090957,16.333224,...,13.134230,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,G1,TCGA-CD-5800-01A
179,16.626200,14.824292,9.646290,17.311402,11.648672,14.836038,19.290417,14.185604,16.489646,14.741389,...,13.475461,0.0,11.040148,0.0,0.0,0.0,0.0,0.0,G1,TCGA-CG-5719-01A
258,16.700326,14.791969,10.173965,18.063640,13.303350,15.178388,19.508899,15.520584,15.745536,15.558286,...,13.933264,0.0,11.819628,0.0,0.0,0.0,0.0,0.0,G1,TCGA-HU-8244-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,16.836165,14.909098,10.470715,16.857157,12.960878,14.552652,18.947518,15.209150,16.418538,15.372440,...,10.038881,0.0,11.923937,0.0,0.0,0.0,0.0,0.0,GX,TCGA-BR-8297-01A
378,17.730010,13.771971,13.388739,17.479320,11.903330,13.810610,18.961728,16.091403,16.579878,15.326050,...,12.574182,0.0,8.335194,0.0,0.0,0.0,0.0,0.0,GX,TCGA-VQ-A91S-01A
344,16.479393,14.211426,0.000000,17.449130,13.599214,13.464333,19.677510,13.738089,15.582241,14.862308,...,13.237325,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,GX,TCGA-SW-A7EA-01A
50,17.446486,14.067786,10.243454,19.095619,11.898451,14.275138,19.348096,14.521417,15.977176,15.750249,...,12.810275,0.0,9.054990,0.0,0.0,0.0,0.0,0.0,GX,TCGA-BR-7197-01A


In [8]:
dfs2.to_pickle('xy_STAD-related_trimmed_sorted.pkl')
dfs2.to_csv('xy_STAD-related_trimmed_sorted.csv')

In [9]:
dfs2 = pd.read_pickle('xy_STAD-related_trimmed_sorted.pkl')
dfs2

,ENSG00000001036,ENSG00000002330,ENSG00000003400,ENSG00000003402,ENSG00000006831,ENSG00000007038,ENSG00000007062,ENSG00000007171,ENSG00000007306,ENSG00000007908,...,ENSG00000278910,ENSG00000280924,ENSG00000280997,ENSG00000281183,ENSG00000281189,ENSG00000281202,ENSG00000281406,ENSG00000281450,neoplasm_histologic_grade,submitter_id.samples
318,20.219629,19.121479,17.287974,16.630257,18.423464,19.828220,19.216307,15.708156,14.225520,12.588927,...,11.962227,13.237309,8.099240,15.439994,12.557625,0.000000,14.470708,0.000000,G1,TCGA-KB-A93H-01A
317,18.819095,17.963353,16.261201,18.064107,18.742024,14.194581,17.918124,16.411844,13.153777,17.957511,...,0.000000,11.723755,10.902820,15.902137,11.159664,0.000000,13.233482,8.801656,G1,TCGA-KB-A93G-01A
132,19.807727,19.515146,17.236227,16.513151,17.901542,15.226351,16.193370,15.621697,17.307695,13.244725,...,0.000000,13.746728,8.880671,13.426548,14.626484,0.000000,16.505431,0.000000,G1,TCGA-CD-5800-01A
179,18.941061,19.171449,16.252021,17.308548,18.033843,18.545737,18.558169,13.269680,15.130262,16.910450,...,0.000000,13.519727,0.000000,15.516455,12.852409,7.879561,16.408713,8.441310,G1,TCGA-CG-5719-01A
258,19.919778,18.456981,17.296716,16.551752,17.765063,10.902787,18.113180,16.785000,18.713718,11.629097,...,0.000000,12.874540,9.264021,15.442020,13.462183,0.000000,14.359035,0.000000,G1,TCGA-HU-8244-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,18.749838,18.851856,16.423086,17.029071,17.873540,17.380176,16.748492,12.067423,10.471021,11.404887,...,11.912352,11.394259,7.374729,15.455789,12.198967,7.031628,13.265510,10.392719,GX,TCGA-BR-8297-01A
378,20.160689,17.841738,17.806221,17.092400,17.932093,13.823015,19.435108,16.823552,15.717842,14.174856,...,11.641512,10.708630,11.487912,14.583587,13.082237,0.000000,15.675195,10.314506,GX,TCGA-VQ-A91S-01A
344,19.631296,18.765269,17.731691,17.073518,18.011159,19.196995,13.348575,17.490483,14.895278,9.664963,...,13.597188,12.146249,0.000000,15.329461,9.124790,0.000000,14.680185,10.784776,GX,TCGA-SW-A7EA-01A
50,19.331497,18.370031,16.306453,17.176000,17.556664,17.652474,18.865459,12.446429,17.184552,14.755209,...,10.363710,12.166602,0.000000,14.522243,13.625953,9.794749,14.882827,0.000000,GX,TCGA-BR-7197-01A


### 1-2. Extracting dataset X & Y

In [16]:
df_X = dfs2.drop(labels=['neoplasm_histologic_grade','submitter_id.samples'], axis=1)
df_X

,ENSG00000001084,ENSG00000001460,ENSG00000002745,ENSG00000003147,ENSG00000003987,ENSG00000004139,ENSG00000004478,ENSG00000004660,ENSG00000004700,ENSG00000004777,...,ENSG00000281843,ENSG00000281849,ENSG00000281884,ENSG00000281905,ENSG00000281920,ENSGR0000167393,ENSGR0000223511,ENSGR0000223773,ENSGR0000227159,ENSGR0000264510
318,17.741417,14.171273,11.285698,18.566679,13.264547,13.822387,18.764541,14.411721,15.706475,15.462137,...,0.0,10.950622,13.087562,0.0,10.330718,0.0,0.0,0.0,0.0,0.0
317,16.358518,13.730371,10.385346,17.389039,13.744497,14.838198,17.894246,14.631428,17.327174,15.896477,...,0.0,13.523648,12.573802,0.0,8.818965,0.0,0.0,0.0,0.0,0.0
132,16.668176,15.105117,10.441765,17.441566,11.878497,14.131243,19.423980,15.489673,15.090957,16.333224,...,0.0,13.733637,13.134230,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
179,16.626200,14.824292,9.646290,17.311402,11.648672,14.836038,19.290417,14.185604,16.489646,14.741389,...,0.0,10.076133,13.475461,0.0,11.040148,0.0,0.0,0.0,0.0,0.0
258,16.700326,14.791969,10.173965,18.063640,13.303350,15.178388,19.508899,15.520584,15.745536,15.558286,...,0.0,13.439655,13.933264,0.0,11.819628,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,16.836165,14.909098,10.470715,16.857157,12.960878,14.552652,18.947518,15.209150,16.418538,15.372440,...,0.0,12.222235,10.038881,0.0,11.923937,0.0,0.0,0.0,0.0,0.0
378,17.730010,13.771971,13.388739,17.479320,11.903330,13.810610,18.961728,16.091403,16.579878,15.326050,...,0.0,11.758787,12.574182,0.0,8.335194,0.0,0.0,0.0,0.0,0.0
344,16.479393,14.211426,0.000000,17.449130,13.599214,13.464333,19.677510,13.738089,15.582241,14.862308,...,0.0,13.269743,13.237325,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
50,17.446486,14.067786,10.243454,19.095619,11.898451,14.275138,19.348096,14.521417,15.977176,15.750249,...,0.0,12.842692,12.810275,0.0,9.054990,0.0,0.0,0.0,0.0,0.0


In [17]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 318 to 66
Columns: 8430 entries, ENSG00000001084 to ENSGR0000264510
dtypes: float64(8430)
memory usage: 26.2 MB


In [18]:
df_Y = dfs2[['neoplasm_histologic_grade']]
df_Y

,neoplasm_histologic_grade
318,G1
317,G1
132,G1
179,G1
258,G1
...,...
82,GX
378,GX
344,GX
50,GX


In [19]:
df_Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 318 to 66
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   neoplasm_histologic_grade  407 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [20]:
df_Y.shape

(407, 1)

In [21]:
print('G1:  ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G1']))
print('G2: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G2']))
print('G3: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G3']))
print('GX:   ', len(df_Y[df_Y['neoplasm_histologic_grade']=='GX']))

G1:   10
G2:  150
G3:  238
GX:    9


In [22]:
# NLL Loss 계산 시 문제가 생겨 label 차원은 평범한 1D가 되게 바꿈 
df_Y = dfs2['neoplasm_histologic_grade']
df_Y

318    G1
317    G1
132    G1
179    G1
258    G1
       ..
82     GX
378    GX
344    GX
50     GX
66     GX
Name: neoplasm_histologic_grade, Length: 407, dtype: object

In [23]:
df_Y.shape, len(df_Y)

((407,), 407)

### 1-3. Splitting dataset into train, val, test 

In [24]:
# X, Y 데이터를 약 7 : 2 : 1 의 비율로 나눔
df_X_train = pd.concat([df_X[0: 7], df_X[ 10:115], df_X[160:327], df_X[398:404]])
df_X_val   = pd.concat([df_X[7: 9], df_X[115:145], df_X[327:375], df_X[404:406]])
df_X_test  = pd.concat([df_X[9:10], df_X[145:160], df_X[375:398], df_X[406:407]])
df_Y_train = pd.concat([df_Y[0: 7], df_Y[ 10:115], df_Y[160:327], df_Y[398:404]])
df_Y_val   = pd.concat([df_Y[7: 9], df_Y[115:145], df_Y[327:375], df_Y[404:406]])
df_Y_test  = pd.concat([df_Y[9:10], df_Y[145:160], df_Y[375:398], df_Y[406:407]])

In [25]:
len(df_Y_train)+len(df_Y_val)+len(df_Y_test)

407

In [26]:
# X 데이터 : pandas dataframe 형식에서 numpy array 형식으로 변환
X_train = df_X_train.to_numpy(dtype='float32')
X_val = df_X_val.to_numpy(dtype='float32')
X_test = df_X_test.to_numpy(dtype='float32')
X_train.shape, X_train[0].dtype

((285, 8430), dtype('float32'))

In [27]:
# Y 데이터 : 문자에서 정수로 변환 G1->1, G2->2, G3->3, GX->0
Y_train = df_Y_train.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_val = df_Y_val.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_test = df_Y_test.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_train.shape, Y_train[0].dtype

((285,), dtype('int8'))

### 1-4. Save datasets in 3 different ways

In [68]:
# 1. train:val:test 가 7:2:1 로 나뉜 데이터를 전부 가지고 있는 파일
hf = h5py.File('STAD_Dataset_split_in_three.h5', 'w')
hf.create_dataset('X_train', data=X_train)
hf.create_dataset('X_val', data=X_val)
hf.create_dataset('X_test', data=X_test)
hf.create_dataset('Y_train', data=Y_train)
hf.create_dataset('Y_val', data=Y_val)
hf.create_dataset('Y_test', data=Y_test)
hf.close()

In [28]:
X_Train = np.concatenate([X_train, X_test])
X_Train.shape

(325, 8430)

In [29]:
Y_Train = np.concatenate([Y_train, Y_test])
Y_Train.shape

(325,)

In [30]:
# 2. 위 train+test 를 합친 전체의 80% 인 Train 데이터파일 
hf = h5py.File('STAD_Dataset_limit_Train.h5', 'w')
hf.create_dataset('X_Train', data=X_Train)
hf.create_dataset('Y_Train', data=Y_Train)
hf.close()

In [31]:
# 3. 나머지 20% 만 가진 Test 데이터파일 
hf = h5py.File('STAD_Dataset_limit_Test.h5', 'w')
hf.create_dataset('X_Test', data=X_val)
hf.create_dataset('Y_Test', data=Y_val)
hf.close()

### 1-5. Dataset Validation Check

In [32]:
hf = h5py.File('STAD_Dataset_split_in_three.h5', 'r')
# hf.keys()
X_train = np.array(hf.get('X_train'))
X_val = np.array(hf.get('X_val'))
X_test = np.array(hf.get('X_test'))
Y_train = np.array(hf.get('Y_train'))
Y_val = np.array(hf.get('Y_val'))
Y_test = np.array(hf.get('Y_test'))

print('X_train.shape : ', X_train.shape)
print('X_val.shape   : ', X_val.shape)
print('X_test.shape  : ', X_test.shape)
print('Y_train.shape : ', Y_train.shape)
print('Y_val.shape   : ', Y_val.shape)
print('Y_test.shape  : ', Y_test.shape)

hf.close()

X_train.shape :  (285, 843)
X_val.shape   :  (82, 843)
X_test.shape  :  (40, 843)
Y_train.shape :  (285,)
Y_val.shape   :  (82,)
Y_test.shape  :  (40,)


In [33]:
hf = h5py.File('STAD_Dataset_Train.h5', 'r')

X_Train = np.array(hf.get('X_Train'))
Y_Train = np.array(hf.get('Y_Train'))
print(hf.keys())
print('X_Train.shape : ', X_Train.shape)
print('Y_Train.shape : ', Y_Train.shape)
print('X_Train' in hf.keys())

hf.close()

<KeysViewHDF5 ['X_Train', 'Y_Train']>
X_Train.shape :  (325, 843)
Y_Train.shape :  (325,)
True


In [34]:
type(X_Train)

numpy.ndarray

In [35]:
hf = h5py.File('STAD_Dataset_Test.h5', 'r')
# hf.keys()
X_Test = np.array(hf.get('X_Test'))
Y_Test = np.array(hf.get('Y_Test'))

print('X_Test.shape  : ', X_Test.shape)
print('Y_Test.shape  : ', Y_Test.shape)

hf.close()

X_Test.shape  :  (82, 843)
Y_Test.shape  :  (82,)


In [92]:
X_Train[0].dtype, Y_Train[0].dtype

(dtype('float32'), dtype('int8'))

## 2. Implementing the Backbone of Transformer-based Classifier

In [32]:
class STAD_Dataset(T.utils.data.Dataset):
    def __init__(self, src_file):
        hf = h5py.File(src_file, 'r')
        if 'X_Train' in hf.keys() :
            X = np.array(hf.get('X_Train')).astype('float32')
            Y = np.array(hf.get('Y_Train')).astype('int8')
        else :
            X = np.array(hf.get('X_Test')).astype('float32')
            Y = np.array(hf.get('Y_Test')).astype('int8')
        hf.close()

        self.x_data = T.tensor(X, dtype=T.float32).to(device)
        self.y_data = T.tensor(Y, dtype=T.int64).to(device)  

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx] 
        return x, y  

In [33]:
# 오리지널 트랜스포머의 embedding layer를 대체하는 특수 레이어 함수
# 데이터가 텍스트가 아닌 실수일 때에는 기존과는 다른 방식이 필요
class Special_Embedding_Layer(T.nn.Module):

    class Core(T.nn.Module):
        def __init__(self, n):
            super().__init__()
            self.weights = T.nn.Parameter(T.zeros((n, 1), dtype=T.float32))
            self.biases = T.nn.Parameter(T.tensor(n, dtype=T.float32))
            lim = 0.01
            T.nn.init.uniform_(self.weights, -lim, lim)
            T.nn.init.zeros_(self.biases)
      
        def forward(self, x):
            wx = T.mm(x, self.weights.t())
            v = T.add(wx, self.biases)
            return v
    

    def __init__(self, n_in, n_out):
        super().__init__()
        self.n_in = n_in; self.n_out = n_out
        if n_out % n_in != 0:
            print("Warning: n_out ought be divisible by n_in")
        n = n_out // n_in
        self.lst_modules = T.nn.ModuleList([Special_Embedding_Layer.Core(n) for i in range(n_in)])


    def forward(self, x):
        lst_nodes = []
        for i in range(self.n_in):
            xi = x[:,i].reshape(-1, 1)
            oupt = self.lst_modules[i](xi)
            lst_nodes.append(oupt)

        # for i in range(1, self.n_in):
        #     result = T.cat((result, lst_nodes[i]), 1)
        # 위와 같이 하면 이상하게 두 번째 것만 붙고 나머지는 안 되어서 할 수 없이 나눠서 함
        result = T.cat((lst_nodes[0], lst_nodes[1]), 1)
        for i in range(2, self.n_in):
            result = T.cat((result, lst_nodes[i]), 1)
        result = result.reshape(-1, self.n_out)

        return result

In [34]:
class PositionalEncoding(T.nn.Module):  
    def __init__(self, d_model: int, dropout: float=0.1, max_len: int=5000):
        super(PositionalEncoding, self).__init__()  
        self.dropout = T.nn.Dropout(p=dropout)
        pe = T.zeros(max_len, d_model)  
        position = T.arange(0, max_len, dtype=T.float).unsqueeze(1)
        div_term = T.exp(T.arange(0, d_model, 2).float() * (-np.log(10_000.0) / d_model))
        pe[:, 0::2] = T.sin(position * div_term)
        pe[:, 1::2] = T.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)  # allows state-save

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [35]:
class Transformer_Classifier(T.nn.Module):  # 843-26976-Transformer-10-4
    def __init__(self):
        super(Transformer_Classifier, self).__init__() 
        # numeric pseudo-embedding, dim=32
        self.embed = Special_Embedding_Layer(8430, 269760)  # 843 input features, each goes to 32 
        self.pos_enc = PositionalEncoding(32, dropout=0.00)  # positional embedding
        self.enc_layer = T.nn.TransformerEncoderLayer(d_model=32, nhead=2, dim_feedforward=32, batch_first=True)  # d_model divisible by nhead
        self.trans_enc = T.nn.TransformerEncoder(self.enc_layer, num_layers=2)  # 6 layers default
        # STAD Dataset has 843 input features per each sample
        self.fc1 = T.nn.Linear(32*8430, 100)  # 10 hidden nodes
        self.fc2 = T.nn.Linear(100, 4)    # 4 classes

    def forward(self, x):
        # x = 843 input features, fixed length
        z = self.embed(x)  # 843 inputs to 26976 embeds 
        z = z.reshape(-1, 8430, 32)  # bat seq embed 
        z = self.pos_enc(z) 
        z = self.trans_enc(z) 
        z = z.reshape(-1, 32*8430)  # torch.Size([bs, xxx])
        z = T.tanh(self.fc1(z))
        z = T.log_softmax(self.fc2(z), dim=1)  # NLLLoss()
        return z

In [37]:
net = Transformer_Classifier().to(device)
net

Transformer_Classifier(
  (embed): Special_Embedding_Layer(
    (lst_modules): ModuleList(
      (0): Core()
      (1): Core()
      (2): Core()
      (3): Core()
      (4): Core()
      (5): Core()
      (6): Core()
      (7): Core()
      (8): Core()
      (9): Core()
      (10): Core()
      (11): Core()
      (12): Core()
      (13): Core()
      (14): Core()
      (15): Core()
      (16): Core()
      (17): Core()
      (18): Core()
      (19): Core()
      (20): Core()
      (21): Core()
      (22): Core()
      (23): Core()
      (24): Core()
      (25): Core()
      (26): Core()
      (27): Core()
      (28): Core()
      (29): Core()
      (30): Core()
      (31): Core()
      (32): Core()
      (33): Core()
      (34): Core()
      (35): Core()
      (36): Core()
      (37): Core()
      (38): Core()
      (39): Core()
      (40): Core()
      (41): Core()
      (42): Core()
      (43): Core()
      (44): Core()
      (45): Core()
      (46): Core()
      (47): Core()
      (

In [39]:
import torchinfo
batchsize = 10
torchinfo.summary(net, input_size=(batchsize, 8430))

Layer (type:depth-idx)                             Output Shape              Param #
Transformer_Classifier                             [10, 4]                   6,464
├─Special_Embedding_Layer: 1-1                     [10, 26976]               --
│    └─ModuleList: 2-1                             --                        --
│    │    └─Core: 3-1                              [10, 32]                  33
│    │    └─Core: 3-2                              [10, 32]                  33
│    │    └─Core: 3-3                              [10, 32]                  33
│    │    └─Core: 3-4                              [10, 32]                  33
│    │    └─Core: 3-5                              [10, 32]                  33
│    │    └─Core: 3-6                              [10, 32]                  33
│    │    └─Core: 3-7                              [10, 32]                  33
│    │    └─Core: 3-8                              [10, 32]                  33
│    │    └─Core: 3-9           

## 3. Training model

### 3-1. Lodaing Dataset

In [40]:
# def main():
# 0. setup
print("\nBegin Transformer Classifier with 407 STAD samples ")
np.random.seed(1)  
T.manual_seed(1)  

# 1. create Dataset
print("\nCreating 325 training dataset from an h5 file")
train_file = "./STAD_Dataset_limit_Train.h5"
train_ds = STAD_Dataset(train_file)

print("\nCreating 82 testdDataset from an h5 file")
test_file = "./STAD_Dataset_limit_Test.h5"
test_ds = STAD_Dataset(test_file)

bat_size = 10
train_ldr = T.utils.data.DataLoader(train_ds, batch_size=bat_size, shuffle=True)


Begin Transformer Classifier with 407 STAD samples 

Creating 325 training dataset from an h5 file

Creating 82 testdDataset from an h5 file


### 3-2. Training model

In [ ]:
print("\nCreating Transformer network ")
net = Transformer_Classifier().to(device)
print("\nPreparation for model training complete.")

max_epochs = 100
ep_log_interval = 1
lrn_rate = 0.0001

loss_func = T.nn.NLLLoss()  # assumes log-softmax()
# optimizer = T.optim.SGD(net.parameters(), lr=lrn_rate)
optimizer = T.optim.Adam(net.parameters(), lr=lrn_rate)

print("\nbat_size = %3d " % bat_size)
print("loss = " + str(loss_func))
print("optimizer = ", optimizer)
# print("lrn_rate = %0.5f " % lrn_rate)
print("max_epochs = %3d " % max_epochs)

print("\nStarting training")
net.train()  # set mode
for epoch in range(0, max_epochs):
    ep_loss = 0.0  # for one full epoch
    for (batch_idx, batch) in enumerate(train_ldr):
        (X, y) = batch  # X = pixels, y = target labels
        optimizer.zero_grad()
        oupt = net(X)
        loss_val = loss_func(oupt, y)  # a tensor
        ep_loss += loss_val.item()  # accumulate
        loss_val.backward()  # compute grads
        optimizer.step()     # update weights
    if epoch % ep_log_interval == 0:
        print("epoch = %4d   |   loss = %9.4f" % (epoch, ep_loss))
        net.eval()

print("Done ") 


Creating Transformer network 

Preparation for model training complete.

bat_size =  10 
loss = NLLLoss()
optimizer =  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
max_epochs = 100 

Starting training
epoch =    0   |   loss =   30.9799
epoch =    1   |   loss =   29.2348
epoch =    2   |   loss =   29.2253
epoch =    3   |   loss =   28.9090
epoch =    4   |   loss =   28.9132
epoch =    5   |   loss =   29.1857
epoch =    6   |   loss =   28.7126
epoch =    7   |   loss =   28.6448
epoch =    8   |   loss =   28.5000
epoch =    9   |   loss =   28.4572
epoch =   10   |   loss =   28.5524
epoch =   11   |   loss =   28.4131
epoch =   12   |   loss =   28.3397
epoch =   13   |   loss =   28.3216
epoch =   14   |   loss =   28.2849
epoch =   15   |   loss =   28.2622
epoch =   16   |   loss =   28.2519
epoch =   17   |   loss =   28.5548
epoch =   18   

## 4. Evaluating model accuracy

In [61]:
def accuracy(model, ds):
  # assumes model.eval()
  # item-by-item version
    n_correct = 0; n_wrong = 0
    for i in range(len(ds)):
        X = ds[i][0].reshape(1,-1)  # make it a batch
        Y = ds[i][1].reshape(1)  # 0 1 or 2, 1D
        with T.no_grad():
            oupt = model(X)  # logits form

        big_idx = T.argmax(oupt)  # 0 or 1 or 2
        if big_idx == Y:
            n_correct += 1
        else:
            n_wrong += 1

    acc = (n_correct * 1.0) / (n_correct + n_wrong)
    return acc

In [62]:
print("\nComputing model accuracy")
net.eval()
acc_train = accuracy(net, train_ds)  # item-by-item
print("Accuracy on training data = %0.4f" % acc_train)

net.eval()
acc_test = accuracy(net, test_ds) 
print("Accuracy on test data = %0.4f" % acc_test)


Computing model accuracy
Accuracy on training data = 0.5846
Accuracy on test data = 0.5854


## 5. Inferrence

In [ ]:
X = np.array([[-1, 0.30,  0,0,1,  0.5000]], dtype=np.float32)
X = T.tensor(X, dtype=T.float32).to(device) 

with T.no_grad():
    logits = net(X)  # do not sum to 1.0
probs = T.exp(logits)  # sum to 1.0
probs = probs.numpy()  # numpy vector prints better
np.set_printoptions(precision=4, suppress=True)
print(probs)

## 6. Save model

In [64]:
print("\nSaving trained model state")
file_name = "./Model_save/SGD_float16.pt"
T.save(net.state_dict(), file_name)  


Saving trained model state


In [ ]:
# # later
# if __name__ == "__main__":
#   main()